In [1]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_groq import ChatGroq
# from ragas import evaluate
# from ragas.embeddings import LangchainEmbeddingsWrapper
# from ragas.llms import LangchainLLMWrapper
# from ragas.metrics import answer_relevancy, faithfulness, context_recall, context_precision
from dotenv import load_dotenv
import os
# from datasets import Dataset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support
import seaborn as sns
import matplotlib.pyplot as plt
# import numpy as np
import pandas as pd
# import requests
import time
from pathlib import Path
from main import main

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
from rouge_score import rouge_scorer
from bert_score import score

c:\Users\krisr\DataSci210_MedicationDeprescriber\src\extraction.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krisr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krisr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\krisr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-

In [2]:
def evaluate_multiclass_classification(y_true, y_pred, class_labels):
    """
    Evaluates a multiclass classification model.
    
    y_true: Ground truth labels
    y_pred: Predicted labels from the classifier
    class_labels: List of class names
    
    Returns a dictionary of accuracy, precision, recall, F1, and confusion matrix.
    """
    
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Precision, Recall, F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')  # macro-averaging
    
    # Classification report (optional detailed breakdown for each class)
    class_report = classification_report(y_true, y_pred, target_names=class_labels)
    
    # Confusion Matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    # Print evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Macro Avg): {precision:.4f}")
    print(f"Recall (Macro Avg): {recall:.4f}")
    print(f"F1 Score (Macro Avg): {f1:.4f}")
    print("\nClassification Report:\n", class_report)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Return metrics for further analysis if needed
    return {
        "accuracy": accuracy,
        "precision_macro": precision,
        "recall_macro": recall,
        "f1_macro": f1,
        "confusion_matrix": conf_matrix
    }


In [ ]:
# # ragas score
# load_dotenv()

# groq = os.getenv("groq_key")

# evaluator_llm = LangchainLLMWrapper(ChatGroq(temperature=0, model="llama-3.1-70b-versatile", api_key=groq))
# evaluator_embeddings = LangchainEmbeddingsWrapper(HuggingFaceEmbeddings(model_name="NeuML/pubmedbert-base-embeddings"))

# def calculate_RAGAS_answer(generated_answer, gold_answer, question, context):
#   data = {'question': [question],
#           'contexts': [context],
#           'answer': [generated_answer],
#           'ground_truth': [gold_answer]}
#   dataset = Dataset.from_dict(data)
#   score = evaluate(dataset, llm=evaluator_llm, embeddings=evaluator_embeddings, metrics = [faithfulness, answer_relevancy, context_precision, context_recall]).to_pandas()
#   faithfulness_score = score['faithfulness'].iloc[0]
#   answer_relevancy_score = score['answer_relevancy'].iloc[0]
#   context_precision_score = score['context_precision'].iloc[0]
#   context_recall_score = score['context_recall'].iloc[0]
#   return faithfulness_score, answer_relevancy_score, context_precision_score, context_recall_score

In [ ]:
# from extraction import llmAgent
# from ppi_deprescribe import merge_results

# llm_agent = llmAgent(groq_key=groq, data_path=path)

# data = pd.read_csv(path / 'LabeledResponses_ReturnedResponse.csv')

# ragas_results_df = pd.DataFrame(columns=['key', 'ragas_score'])

# for key in data['key']:
#     print(f"Starting key {key}")
#     start = time.time()

#     temp = data[data['key'] == key]
#     temp = temp.reset_index()

#     results_dict = {
#         "diagnosis_dict": llm_agent.extract_diagnosis(encounter_key=key),
#         "encounter_dict": llm_agent.extract_encounter_info(encounter_key=key),
#         "notes_dict": llm_agent.extract_notes(encounter_key=key),
#     }
#     final_dict = merge_results(results_dict=results_dict)

#     final_reasoning = llm_agent.summarize_reasonings(results_dict=results_dict)

#     generated_answer = temp['Reasoning'][0]
#     gold_answer = temp['GS_response'][0]
#     question = "You are a knowledgeable medical provider who specializes in medication management. Given a list of diagnosis and some snippets from patients notes, answer if the patient notes contain any of the diagnosis. Based on the information from the note context, does the patient have any of the following: 1. Mild to moderate esophagitis 2. GERD 3. Peptic Ulcer Disease 4. Upper GI symptoms 5. ICU Stress Ulcer Prophylaxis 6. Barretts Esophagus 7. Chronic NSAID use with bleeding risk 8. Severe esophagitis 9. Documented history of bleeding GI ulcer 10. H pylori infection 11. Explain the reasoning for your answer. Return the answer for each of these as a formatted JSON object with the key being the condition and the value being a boolean value for the first 10.  For the final question, return a string with the reasoning for your answer. Summarize the reasonings from the three sources. You are a knowledgeable medical provider who specializes in medication management. Based on the following json files, please provide a single explanation of the reasoning given by the 'Reasoning' key. Summarize given equal weight to each. Do not add any additional information, only summarize what is given." 
#     context = [final_reasoning]

#     ragas_score = calculate_RAGAS_answer(generated_answer, gold_answer, question, context)

#     print(ragas_score)
#     print(f"{key} took {time.time() - start} seconds to process.")

#     new_row = pd.DataFrame([{'key': key, 'ragas_score': ragas_score}])
#     ragas_results_df = pd.concat([ragas_results_df, new_row], ignore_index=True)

# ragas_results_df.to_csv(path / 'model_results.csv', index=False)


In [ ]:
load_dotenv()

groq = os.getenv("groq_key")
path = Path(os.getenv("data_path"))

data = pd.read_csv(path / 'LabeledResponses.csv')

results_df = pd.DataFrame(columns=['key', 'rec', 'response'])

for key in data['key']:
    start = time.time()
    print(f"Starting key: {key}.")
    response = main(groq, path, key)
    new_row = pd.DataFrame([{'key': key, 'rec': response[0], 'response': response[1]}])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    print(f"{key} took {time.time() - start} seconds to process.")

results_df.to_csv(path / 'model_results.csv', index=False)

In [2]:
load_dotenv()

path = Path(os.getenv("data_path"))

rouge1_scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
rouge2_scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
rougeL_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

smoothing_function = SmoothingFunction().method1

# data = pd.read_csv(path / 'summary_run_results.csv')
# data = pd.read_csv(path / 'summary_with_notes_run_results.csv')
data = pd.read_csv(path / 'baseline_eval.csv')

results_df = pd.DataFrame(columns=['key', 'GS_response', 'ret_reasoning', 'bert_Precision', 'bert_Recall', 'bert_F1',
                                   'rouge1_Precision', 'rouge1_Recall', 'rouge1_F1', 
                                   'rouge2_Precision', 'rouge2_Recall', 'rouge2_F1', 
                                   'rougeL_Precision', 'rougeL_Recall', 'rougeL_F1', 
                                   'METEOR_score', 'bleu_score'
                                   ])

data = data[['key', 'GS_response', 'ret_reasoning']]

for key in data['key']:
    start = time.time()
    temp = data[data['key'] == key].reset_index()

    # Compute BERTScore
    bert_P, bert_R, bert_F1 = score([temp['ret_reasoning'][0]], [temp['GS_response'][0]], lang="en", verbose=False)

    # Compute Rouge scores
    rouge1_scores = rouge1_scorer.score(temp['GS_response'][0], temp['ret_reasoning'][0])
    rouge2_scores = rouge2_scorer.score(temp['GS_response'][0], temp['ret_reasoning'][0])
    rougeL_scores = rougeL_scorer.score(temp['GS_response'][0], temp['ret_reasoning'][0])

    response_tokens = word_tokenize(temp['ret_reasoning'][0])
    gs_response_tokens = word_tokenize(temp['GS_response'][0])

    # Compute METEOR score
    score_from_meteor = meteor_score([gs_response_tokens], response_tokens)

    # Compute Bleu score
    bleu_score = sentence_bleu(gs_response_tokens, response_tokens, smoothing_function=smoothing_function)

    new_row = pd.DataFrame([{'key': key, 'GS_response': temp['GS_response'][0], 'ret_reasoning': temp['ret_reasoning'][0],
                            'bert_Precision': bert_P.item(), 'bert_Recall': bert_R.item(), 'bert_F1': bert_F1.item(), 
                            'rouge1_Precision': rouge1_scores['rouge1'].precision, 'rouge1_Recall': rouge1_scores['rouge1'].recall, 'rouge1_F1': rouge1_scores['rouge1'].fmeasure, 
                            'rouge2_Precision': rouge2_scores['rouge2'].precision, 'rouge2_Recall': rouge2_scores['rouge2'].recall, 'rouge2_F1': rouge2_scores['rouge2'].fmeasure, 
                            'rougeL_Precision': rougeL_scores['rougeL'].precision, 'rougeL_Recall': rougeL_scores['rougeL'].recall, 'rougeL_F1': rougeL_scores['rougeL'].fmeasure, 
                            'METEOR_score': score_from_meteor, 'bleu_score': bleu_score
                            }])
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    print(f"{key} took {time.time() - start} seconds to process.")

# results_df.to_csv(path / 'summary_results.csv', index=False)
# results_df.to_csv(path / 'summary_with_notes_results.csv', index=False)
results_df.to_csv(path / 'baseline_results.csv', index=False)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\krisr\AppData\Local\Temp\ipykernel_25640\509916528.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)


D6253A5CE371EA took 7.657536506652832 seconds to process.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DFAFED1811B871 took 1.948941707611084 seconds to process.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

SyntheticPt4 took 1.4568538665771484 seconds to process.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SyntheticPt5 took 1.4689781665802002 seconds to process.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SyntheticPt6 took 1.9591114521026611 seconds to process.


In [3]:
load_dotenv()

path = Path(os.getenv("data_path"))

# summary_results = pd.read_csv(path / 'summary_results.csv')
# summary_with_notes_results = pd.read_csv(path / 'summary_with_notes_results.csv')
baseline_results = pd.read_csv(path / 'baseline_results.csv')

# summary_results = summary_results.drop(["key", "GS_response", "ret_reasoning"], axis=1)
# summary_with_notes_results = summary_with_notes_results.drop(["key", "GS_response", "ret_reasoning"], axis=1)
baseline_results = baseline_results.drop(["key", "GS_response", "ret_reasoning"], axis=1)

# min_summary_results = summary_results.min()
# max_summary_results = summary_results.max()
# med_summary_results = summary_results.median()
# avg_summary_results = summary_results.mean()

avg_baseline_results = baseline_results.mean()

# print("Summary results:")
# print(f"Minimum scores: \n{min_summary_results}")
# print(f"Maximum scores: \n{max_summary_results}")
# print(f"Median scores: \n{med_summary_results}")
# print(f"Average scores: \n{avg_summary_results}")

# min_summary_with_notes_results = summary_with_notes_results.min()
# max_summary_with_notes_results = summary_with_notes_results.max()
# med_summary_with_notes_results = summary_with_notes_results.median()
# avg_summary_with_notes_results = summary_with_notes_results.mean()

# print("Summary with notes results:")
# print(f"Minimum scores: \n{min_summary_with_notes_results}")
# print(f"Maximum scores: \n{max_summary_with_notes_results}")
# print(f"Median scores: \n{med_summary_with_notes_results}")
# print(f"Average scores: \n{avg_summary_with_notes_results}")

print("Baseline results:")
print(f"Average scores: \n{avg_baseline_results}")

Baseline results:
Average scores: 
bert_Precision      0.834044
bert_Recall         0.838979
bert_F1             0.836331
rouge1_Precision    0.166015
rouge1_Recall       0.281776
rouge1_F1           0.201841
rouge2_Precision    0.028220
rouge2_Recall       0.043931
rouge2_F1           0.033558
rougeL_Precision    0.099091
rougeL_Recall       0.169337
rougeL_F1           0.120264
METEOR_score        0.157428
bleu_score          0.004864
dtype: float64


In [2]:
# load data
data = pd.read_csv(path / 'LabeledResponses_ReturnedResponse.csv')

y_true = data['recommendation']
y_pred = data['Rec_Returned']
class_labels = ['continue', 'deprescribe', 'stop']

# Evaluate the model
metrics = evaluate_multiclass_classification(y_true, y_pred, class_labels)

NameError: name 'pd' is not defined